In [44]:
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import pandas_datareader.data as web

style.use('ggplot')

start = dt.datetime(2015, 1, 1)
end = dt.datetime(2016, 12, 31)

df = web.DataReader('TSLA', "yahoo", start, end)

print(df.head())
web.DataReader('TSLA', "yahoo", start, end).to_csv('inputs_tmp.csv')


                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2014-12-31  223.089996  225.679993  222.250000  222.410004  222.410004   
2015-01-02  222.869995  223.250000  213.259995  219.309998  219.309998   
2015-01-05  214.550003  216.500000  207.160004  210.089996  210.089996   
2015-01-06  210.059998  214.199997  204.210007  211.279999  211.279999   
2015-01-07  213.350006  214.779999  209.779999  210.949997  210.949997   

             Volume  
Date                 
2014-12-31  2402100  
2015-01-02  4764400  
2015-01-05  5368500  
2015-01-06  6261900  
2015-01-07  2968400  


In [52]:

#inputs = []

#inputs.append(df['Date'])

print df.to_json(Date="2014-12-31", date_format='iso') 




TypeError: to_json() got an unexpected keyword argument 'Date'